# CareMap: EHR Companion

## One Model. Three Modules. Maximum Impact on Both Sides of Healthcare.

---

**Kaggle Solve for India with Gemini Competition**

CareMap demonstrates how a single clinical AI model (MedGemma) can enhance healthcare on **both sides**:

| Side | Who | Module | MedGemma Mode |
|------|-----|--------|---------------|
| **Patient Side** | Caregivers, Ayahs, Families | Fridge Sheet Generator | Text Reasoning |
| **Provider Side** | Radiologists | X-ray Triage Queue | **Multimodal** |
| **Provider Side** | Lab/Clinical Staff | HL7 Message Triage | Text Reasoning |

---

## The Problem: Healthcare Information Overload

### On the Patient Side

Family caregivers receive overwhelming amounts of medical information:
- Dense clinical notes with medical jargon
- Multiple medication instructions with complex interactions
- Lab results with unexplained abbreviations
- Follow-up tasks scattered across documents

**In India specifically**, the challenge is compounded by:
- **Rotating Ayahs** (home health workers) who change frequently
- **Language barriers** - Ayahs often cannot read English
- **No EHR infrastructure** - patients carry paper records in bags
- **Overwhelmed doctors** - seeing 125-175 patients per day

### On the Provider Side

Healthcare workers are drowning in data:
- **Radiologists**: 1:100,000 patient ratio in India, 72-hour report delays
- **Lab Staff**: 1000+ HL7 messages per day, critical values buried in routine results
- **No prioritization** - life-threatening findings wait in queue behind normal studies

---

## User Research: Voices from the Field

We interviewed **4 healthcare professionals and caregivers** to understand the real challenges. Their insights directly shaped how MedGemma powers CareMap.

---

### The Need for Simple, Plain-Language Explanations

**Dr. Vinodhini Sriram** (Primary Care Physician, US)

> "For the caregivers, I think it's mainly the medications. They are not going to care about the kidney function... **All they need is that first page which is the medication 8:00 a.m., 12 p.m., whatever it is.**"

> "Absolutely. If the source of this data is reliable, **this is very good.**"

*How MedGemma helps:* Transforms complex sig codes like "PO BID AC" into plain language: "Take by mouth, twice daily, before meals."

---

### The Family Caregiver's Burden

**Sunayana Mann** (Daughter/Family Caregiver)

> "If I had to drive her to the hospital, she cannot tell me all the medications and I might not be able to grab the whole bag. **So easiest would be... I peel the fridge magnet.**"

> "**Task oriented for them** like I don't want them to think. I want to give them instructions and I want them to follow it like nothing extra."

> "**I'm not a medical professional.** I don't know all these things... I need someone to tell me what questions to ask."

*How MedGemma helps:* Generates "Ask the Doctor" prompts so families know what questions matter for their specific situation.

---

### The Reality of Healthcare in India

**Dr. Manini Moudgal** (Pediatrician, Bangalore, India)

> "Given the volumes that we deal with over here, **EHR is a bloody headache**... We usually see **125-175 patients a day.**"

> "The parents don't understand what I'm writing. They just want to know - **when do I give this medicine, how much, and what should I watch for?**"

*How MedGemma helps:* Extracts the essential "what, when, and watch for" from clinical notes, eliminating jargon.

---

### Two Audiences, Two Levels of Detail

**Dr. Gaurav Mishra** (Psychiatrist, Geriatric Focus)

> "**The pages for the ayah need to be even simpler**... for kids with autism sometimes we would use image based chart."

> "I would have **both levels available**... deep dive for family, simple for ayah."

> "The ayah changes every week. **How do you hand off care?** You can't expect verbal instructions to work."

*How MedGemma helps:* Generates separate pages for Ayahs (visual, task-focused) and Family (context, explanations, connections).

---

### Common Themes → CareMap Design Principles

| User Research Insight | CareMap Response |
|----------------------|------------------|
| "All they need is medication times" | Page 1: Medication Schedule with time/food badges |
| "I need someone to tell me what to ask" | MedGemma generates "Ask the Doctor" prompts |
| "Two levels - deep dive and simple" | Separate pages for Ayah vs Family audiences |
| "Ayah changes every week" | Printable poster reduces reliance on verbal handoffs |
| "Don't omit medications" | All 8 medications shown, nothing hidden |
| "Emergency contacts" | Page 5: Who to call when something goes wrong |

---

## The Hypothesis

**A single clinical AI model can serve BOTH sides of healthcare:**

```
                    ┌─────────────────────────────────┐
                    │       MedGemma 1.5 4B-IT        │
                    │   Clinical Foundation Model     │
                    │   (Text + Multimodal)           │
                    └─────────────────────────────────┘
                                   │
              ┌────────────────────┼────────────────────┐
              │                    │                    │
              ▼                    ▼                    ▼
    ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐
    │  Module 1       │  │  Module 2       │  │  Module 3       │
    │  Fridge Sheets  │  │  X-ray Triage   │  │  HL7 Triage     │
    │  (Patient Side) │  │  (Multimodal)   │  │  (Provider Side)│
    └─────────────────┘  └─────────────────┘  └─────────────────┘
           │                    │                    │
           ▼                    ▼                    ▼
    Ayahs & Families     Radiologists        Lab Staff
```

**Why MedGemma?**

MedGemma is specifically trained on clinical text including **Electronic Health Records (EHR)**, making it ideal for:
- Understanding medication sig codes (e.g., "PO BID AC" → "by mouth, twice daily, before meals")
- Interpreting lab result patterns and clinical significance
- Analyzing chest X-rays for findings and urgency
- Translating medical jargon while preserving safety-critical information

---

## Environment Setup

In [ ]:
# Install dependencies for Kaggle environment
!pip install -q transformers>=4.50.0 accelerate>=0.27.0 huggingface-hub>=0.34.0 \
    pydantic>=2.6.0 sentencepiece tqdm lxml


In [ ]:
import json
import os
import sys
from pathlib import Path
from typing import Dict, Any, List
from IPython.display import display, Markdown, HTML, IFrame
from tqdm import tqdm

# --- Kaggle-specific setup ---
# Authenticate with HuggingFace for gated model access
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
hf_token = secrets.get_secret("HUGGINGFACE_TOKEN")
os.environ["HF_TOKEN"] = hf_token

# Dataset is mounted at /kaggle/input/<dataset-name>/
DATASET_DIR = Path("/kaggle/input/caregap-medgemma")
project_root = DATASET_DIR
sys.path.insert(0, str(DATASET_DIR / "src"))

print(f"Project root: {project_root}")
print(f"Python version: {sys.version}")
print(f"HF auth: {'OK' if hf_token else 'MISSING'}")

In [ ]:
# Load MedGemma
from caremap.llm_client import MedGemmaClient

print("Loading MedGemma 1.5 4B-IT...")
print("(This may take 1-2 minutes on first run)")

client = MedGemmaClient()

print(f"\nModel: {client.model_id}")
print(f"Device: {client.device}")
print("Ready!")

---

## Module 1: Patient Portal - Fridge Sheet Generator

### The Use Case: Dadu's Care Team

**Dadu** is an 82-year-old grandfather with:
- Type 2 Diabetes with kidney complications (CKD Stage 3b)
- Congestive Heart Failure
- Atrial Fibrillation (on blood thinners)

He has **8 medications** with complex interactions, **6 lab results** to monitor, and **5 care actions** to track.

His care team includes:
- **Rotating Ayahs** who may change weekly
- **His daughter** who lives in another city
- **His spouse** who is also elderly

CareMap generates **5 printable pages** tailored to each audience:

| Page | Audience | Purpose |
|------|----------|--------|
| 1. Medications | Ayah/Helper | Daily schedule with time/food badges |
| 2. Labs | Family Caregiver | Test results in plain language |
| 3. Care Actions | Family Caregiver | Today/Week/Later task buckets |
| 4. Imaging | Family Caregiver | X-ray findings explained |
| 5. Connections | Both | How meds, labs, and actions connect |

In [ ]:
# Load Dadu's patient data
golden_file = project_root / 'examples' / 'golden_patient_complex.json'

with open(golden_file) as f:
    patient_data = json.load(f)

print(f"Patient: {patient_data['patient']['nickname']}")
print(f"Age: {patient_data['patient']['age_range']}")
print(f"Conditions: {', '.join(patient_data['patient']['conditions_display'])}")
print(f"\nMedications: {len(patient_data['medications'])}")
print(f"Lab Results: {len(patient_data['results'])}")
print(f"Care Gaps: {len(patient_data['care_gaps'])}")

### 1.1 Medication Interpretation

MedGemma transforms complex medication instructions into plain language:

**Input (from EHR):**
```
Warfarin - Take as directed based on INR results
Clinician Notes: Target INR 2.0-3.0 for AFib. Weekly INR checks required.
Interaction Notes: Avoid NSAIDs. Keep vitamin K intake consistent.
```

**Output (for Ayah):**
- **What it does:** Helps prevent dangerous blood clots
- **When:** Evening, same time each day
- **Watch for:** Unusual bleeding, bruising

In [ ]:
from caremap.medication_interpretation import interpret_medication_v3_grounded

print("Interpreting medications with MedGemma...")
print("=" * 60)

# Interpret first 3 medications as examples
for med in tqdm(patient_data['medications'][:3], desc="Medications"):
    result, _ = interpret_medication_v3_grounded(
        client=client,
        medication_name=med['medication_name'],
        sig_text=med['sig_text'],
        clinician_notes=med.get('clinician_notes', ''),
        interaction_notes=med.get('interaction_notes', ''),
    )
    
    print(f"\n{'='*60}")
    print(f"MEDICATION: {med['medication_name']} ({med['timing']})")
    print(f"{'='*60}")
    if 'raw_response' not in result:
        print(f"What this does: {result.get('what_this_does', 'N/A')}")
        print(f"Watch out for: {result.get('watch_out_for', 'N/A')}")

### 1.2 Lab Interpretation

MedGemma explains lab results without numeric values or medical jargon:

**Design Principle:** Caregivers don't need to know "INR = 3.5". They need to know:
- What the test checks
- Whether the result is concerning
- What question to ask the doctor

In [ ]:
from caremap.lab_interpretation import interpret_lab

print("Interpreting lab results with MedGemma...")
print("=" * 60)

for lab in tqdm(patient_data['results'][:3], desc="Labs"):
    result = interpret_lab(
        client=client,
        test_name=lab['test_name'],
        meaning_category=lab['meaning_category'],
        source_note=lab.get('source_note', ''),
    )
    
    print(f"\n{'='*60}")
    print(f"TEST: {lab['test_name']} ({lab['flag']})")
    print(f"{'='*60}")
    print(f"What was checked: {result.get('what_was_checked', 'N/A')}")
    print(f"What it means: {result.get('what_it_means', 'N/A')}")
    print(f"Ask the doctor: {result.get('what_to_ask_doctor', 'N/A')}")

### 1.3 Care Gap Interpretation

MedGemma converts clinical care gaps into actionable tasks:

**Input:** "INR recheck needed - last result was high"

**Output:**
- **Action:** Call clinic to schedule INR blood draw
- **Why:** Blood thinner level needs monitoring
- **Urgency:** This Week

In [ ]:
from caremap.caregap_interpretation import interpret_caregap

print("Interpreting care gaps with MedGemma...")
print("=" * 60)

for gap in tqdm(patient_data['care_gaps'][:3], desc="Care Gaps"):
    result = interpret_caregap(
        client=client,
        item_text=gap['item_text'],
        next_step=gap['next_step'],
        time_bucket=gap['time_bucket'],
    )
    
    print(f"\n{'='*60}")
    print(f"GAP: {gap['item_text']} [{gap['time_bucket']}]")
    print(f"{'='*60}")
    print(f"Time bucket: {result.get('time_bucket', 'N/A')}")
    print(f"Action: {result.get('action_item', 'N/A')}")
    print(f"Next step: {result.get('next_step', 'N/A')}")

### 1.4 Imaging Interpretation

For heart failure patients like Dadu, chest X-rays reveal important information about fluid buildup. MedGemma can explain findings in plain language while connecting them to the patient's medications.

**Safety Constraint:** CareMap never uses the word "cancer" - that's for the oncologist.

In [ ]:
from caremap.imaging_interpretation import interpret_imaging_report

# Load a CT scenario for a heart failure patient
ct_file = project_root / 'examples' / 'golden_imaging_ct.json'
with open(ct_file) as f:
    ct_data = json.load(f)

ct_scenario = ct_data['ct_scenarios'][0]  # CHF patient with shortness of breath

print(f"Study: {ct_scenario['study_type']}")
print(f"Indication: {ct_scenario['clinical_indication']}")
print(f"\nRadiology Report (raw):")
print(f"{ct_scenario['radiology_report']['impression']}")

print("\nInterpreting with MedGemma...")

imaging_result = interpret_imaging_report(
    client=client,
    study_type=ct_scenario['study_type'],
    report_text=ct_scenario['radiology_report']['impression'],
    flag=ct_scenario['flag'],
)

print(f"\n{'='*60}")
print(f"IMAGING INTERPRETATION (Plain Language)")
print(f"{'='*60}")
print(f"What was done: {imaging_result.get('what_was_done', 'N/A')}")
print(f"Key finding: {imaging_result.get('key_finding', 'N/A')}")
print(f"Ask the doctor: {imaging_result.get('what_to_ask_doctor', 'N/A')}")

### 1.5 Connections: The Big Picture

CareMap helps caregivers understand **why** things are connected:

```
Warfarin (blood thinner) ──→ INR Test ──→ Weekly Monitoring
         │                      │
         └── Avoid NSAIDs ──────┘
```

This is especially valuable when explaining care to new Ayahs.

### 1.6 Generate Complete Fridge Sheets (5 Pages)

Now we generate all 5 printable HTML pages:

In [ ]:
from caremap.fridge_sheet_html import (
    generate_medications_page,
    generate_labs_page,
    generate_gaps_page,
    generate_imaging_page,
    generate_connections_page,
    PatientInfo,
)

# Create patient info
patient_info = PatientInfo(
    nickname=patient_data['patient']['nickname'],
    age_range=patient_data['patient']['age_range'],
    conditions=patient_data['patient']['conditions_display']
)

# Output directory (writable on Kaggle)
html_output_dir = Path('/kaggle/working/fridge_sheets')
html_output_dir.mkdir(parents=True, exist_ok=True)

print(f"Generating 5 Concept B pages for {patient_info.nickname}...")
print(f"Output: {html_output_dir}")


In [ ]:
# Page 1: Medications (for Ayah)
print("\n[1/5] Generating MEDICATIONS page (for Ayah)...")

meds_html = generate_medications_page(
    patient=patient_info,
    medications=patient_data['medications'],
    client=client,
    page_num=1,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

med_file = html_output_dir / '1_medications.html'
with open(med_file, 'w') as f:
    f.write(meds_html)
print(f"      Saved: {med_file.name}")

In [ ]:
# Page 2: Labs (for Family)
print("\n[2/5] Generating LABS page (for Family)...")

labs_html = generate_labs_page(
    patient=patient_info,
    results=patient_data['results'],
    client=client,
    page_num=2,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

labs_file = html_output_dir / '2_labs.html'
with open(labs_file, 'w') as f:
    f.write(labs_html)
print(f"      Saved: {labs_file.name}")

In [ ]:
# Page 3: Care Gaps (for Family)
print("\n[3/5] Generating CARE GAPS page (for Family)...")

gaps_html = generate_gaps_page(
    patient=patient_info,
    care_gaps=patient_data['care_gaps'],
    client=client,
    page_num=3,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

gaps_file = html_output_dir / '3_care_gaps.html'
with open(gaps_file, 'w') as f:
    f.write(gaps_html)
print(f"      Saved: {gaps_file.name}")

In [ ]:
# Page 4: Imaging (with X-ray from NIH dataset)
print("\n[4/5] Generating IMAGING page with chest X-ray...")

xray_path = project_root / 'data' / 'nih_chest_xray' / 'demo_images' / 'stat' / '00000032_001.png'
print(f"      Using X-ray: {xray_path.name}")

imaging_html = generate_imaging_page(
    patient=patient_info,
    image_path=str(xray_path) if xray_path.exists() else None,
    client=client,
    page_num=4,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

imaging_file = html_output_dir / '4_imaging.html'
with open(imaging_file, 'w') as f:
    f.write(imaging_html)
print(f"      Saved: {imaging_file.name}")

In [ ]:
# Page 5: Connections (for Both)
print("\n[5/5] Generating CONNECTIONS page (for Both)...")

connections_html = generate_connections_page(
    patient=patient_info,
    medications=patient_data['medications'],
    results=patient_data['results'],
    care_gaps=patient_data['care_gaps'],
    contacts=patient_data['contacts'],
    page_num=5,
    total_pages=5,
    progress_callback=lambda c, t, m: print(f"      [{c}/{t}] {m}")
)

connections_file = html_output_dir / '5_connections.html'
with open(connections_file, 'w') as f:
    f.write(connections_html)
print(f"      Saved: {connections_file.name}")

print("\n" + "=" * 60)
print("All 5 fridge sheet pages generated!")
print("=" * 60)

In [ ]:
# Preview Page 1: Medications (for Ayah)
print("Preview: Page 1 - Medications (for Ayah)")
display(HTML(med_file.read_text()))

In [ ]:
# Preview Page 2: Labs (for Family)
print("Preview: Page 2 - Labs (for Family)")
display(HTML(labs_file.read_text()))

In [ ]:
# Preview Page 3: Care Gaps (for Family)
print("Preview: Page 3 - Care Gaps (for Family)")
display(HTML(gaps_file.read_text()))

In [ ]:
# Preview Page 4: Imaging (for Family)
print("Preview: Page 4 - Imaging (for Family)")
display(HTML(imaging_file.read_text()))

In [ ]:
# Preview Page 5: Connections (for Both)
print("Preview: Page 5 - Connections (for Both)")
display(HTML(connections_file.read_text()))

### 1.7 Multilingual Support: Bengali Fridge Sheet

For Ayahs who cannot read English, CareMap can translate the fridge sheet using Meta's NLLB-200 model:

In [ ]:
from caremap.translation import NLLBTranslator

print("Loading NLLB-200 translator...")
translator = NLLBTranslator()
print("Translator ready!")

# Example: Translate medication instructions to Bengali
sample_text = "This medicine helps control your heart rate. Take it every day at the same time. Do NOT stop taking it suddenly."

print(f"\nOriginal (English):")
print(f"  {sample_text}")

bengali = translator.translate_to(sample_text, "ben_Beng")
print(f"\nBengali (for Ayah):")
print(f"  {bengali}")

### 1.8 Bengali Fridge Sheets: Full Page Translation

Now we translate the **complete fridge sheet HTML pages** to Bengali using NLLB-200. This demonstrates the real-world value: an Ayah receives the **same professionally formatted fridge sheet**, but with all instructions in her language.

**What gets translated:**
- Page headings and labels
- MedGemma-generated explanations (why it matters, watch for)
- Action items and next steps
- Time bucket labels (Today, This Week, Later)

**What stays in English (safety-critical):**
- Medication names (Metformin, Warfarin, etc.)
- Dosage amounts (500mg, etc.)
- MedGemma badge

In [ ]:
from caremap.html_translator import translate_fridge_sheet_html
import threading, time

print("Translating Medication Schedule to Bengali (\u09ac\u09be\u0982\u09b2\u09be)...")
print("=" * 60)

translate_result = {}
translate_error = [None]

def run_med_translation():
    try:
        translate_result['html'] = translate_fridge_sheet_html(
            html_content=meds_html,
            translator=translator,
            target_lang="ben_Beng",
            progress_callback=lambda c, t, m: print(f"  [{c}/{t}] {m}")
        )
    except Exception as e:
        translate_error[0] = e

thread = threading.Thread(target=run_med_translation)
thread.start()

with tqdm(desc="Medication translation (NLLB-200)", unit="s",
          bar_format="{desc}: {elapsed} elapsed |{bar}|") as pbar:
    while thread.is_alive():
        time.sleep(1)
        pbar.update(1)

thread.join()
if translate_error[0]:
    raise translate_error[0]

bengali_meds_html = translate_result['html']

bengali_med_file = html_output_dir / '1_medications_bn.html'
with open(bengali_med_file, 'w', encoding='utf-8') as f:
    f.write(bengali_meds_html)

print(f"\nSaved: {bengali_med_file.name}")
print("\nPreview: Bengali Medication Schedule (for Ayah)")
display(HTML(bengali_meds_html))

In [ ]:
print("Translating Care Actions to Bengali (\u09ac\u09be\u0982\u09b2\u09be)...")
print("=" * 60)

translate_result = {}
translate_error = [None]

def run_gaps_translation():
    try:
        translate_result['html'] = translate_fridge_sheet_html(
            html_content=gaps_html,
            translator=translator,
            target_lang="ben_Beng",
            progress_callback=lambda c, t, m: print(f"  [{c}/{t}] {m}")
        )
    except Exception as e:
        translate_error[0] = e

thread = threading.Thread(target=run_gaps_translation)
thread.start()

with tqdm(desc="Care gaps translation (NLLB-200)", unit="s",
          bar_format="{desc}: {elapsed} elapsed |{bar}|") as pbar:
    while thread.is_alive():
        time.sleep(1)
        pbar.update(1)

thread.join()
if translate_error[0]:
    raise translate_error[0]

bengali_gaps_html = translate_result['html']

bengali_gaps_file = html_output_dir / '3_care_gaps_bn.html'
with open(bengali_gaps_file, 'w', encoding='utf-8') as f:
    f.write(bengali_gaps_html)

print(f"\nSaved: {bengali_gaps_file.name}")
print("\nPreview: Bengali Care Actions (for Family)")
display(HTML(bengali_gaps_html))

---

## Module 2: Radiology Triage (Multimodal)

### The Problem

| Metric | India | Impact |
|--------|-------|--------|
| Radiologist-to-patient ratio | 1:100,000 | Severe shortage |
| Average X-ray report delay | 72 hours | Critical findings missed |
| Daily imaging volume | 500+ studies/radiologist | Burnout, errors |

### The Solution: MedGemma Multimodal Triage

MedGemma analyzes chest X-rays and assigns priority:

| Priority | Review Time | Examples |
|----------|-------------|----------|
| STAT | < 1 hour | Pulmonary edema, pneumothorax |
| SOON | < 24 hours | Cardiomegaly, consolidation |
| ROUTINE | 48-72 hours | Normal, minor findings |

**Key Principle:** AI prioritizes the worklist. Radiologists review ALL images.

In [ ]:
# Free text-only client to make GPU room for multimodal
import gc, torch
del client
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Load MedGemma with multimodal support (reuses same model weights)
print("Loading MedGemma with multimodal support...")
multimodal_client = MedGemmaClient(enable_multimodal=True)
client = multimodal_client  # Reuse for any remaining text cells
print(f"Device: {multimodal_client.device}")
print("Ready for image analysis!")

In [ ]:
# Load sample X-ray from NIH dataset
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

demo_images_dir = project_root / 'data' / 'nih_chest_xray' / 'demo_images'
manifest_path = project_root / 'data' / 'nih_chest_xray' / 'sample_manifest.csv'
manifest = pd.read_csv(manifest_path)

# Display a STAT case
stat_image = manifest[manifest['priority'] == 'STAT'].iloc[0]
image_path = demo_images_dir / 'stat' / stat_image['image_id']

print(f"STAT Case: {stat_image['image_id']}")
print(f"Patient: {stat_image['patient_age']} year old {stat_image['patient_gender']}")
print(f"Ground Truth: {stat_image['findings']}")

img = Image.open(image_path)
plt.figure(figsize=(8, 8))
plt.imshow(img, cmap='gray')
plt.title(f"Chest X-ray: {stat_image['image_id']}")
plt.axis('off')
plt.show()

In [ ]:
import threading
import time
from caremap.radiology_triage import analyze_xray

print("Analyzing X-ray with MedGemma multimodal...")
print("(Single LLM call - typically takes 1-2 minutes on MPS)")

xray_result = {}
xray_error = [None]

def run_xray_analysis():
    try:
        xray_result.update(analyze_xray(
            client=multimodal_client,
            image_path=str(image_path),
            patient_age=int(stat_image['patient_age']),
            patient_gender=stat_image['patient_gender']
        ))
    except Exception as e:
        xray_error[0] = e

thread = threading.Thread(target=run_xray_analysis)
thread.start()

with tqdm(desc="X-ray Triage (MedGemma inference)", unit="s", bar_format="{desc}: {elapsed} elapsed |{bar}|") as pbar:
    while thread.is_alive():
        time.sleep(1)
        pbar.update(1)

thread.join()
if xray_error[0]:
    raise xray_error[0]

result = xray_result
priority_emoji = {'STAT': '\U0001F534', 'SOON': '\U0001F7E1', 'ROUTINE': '\U0001F7E2'}
priority = result.get('priority', 'STAT')

print(f"\n{priority_emoji.get(priority, '')} MedGemma Triage Result: {priority}")
print(f"Confidence: {result.get('confidence', 0):.0%}")
print(f"\nFindings:")
for finding in result.get('findings', []):
    print(f"  - {finding}")
print(f"\nGround Truth: {stat_image['findings']}")

---

## Module 3: HL7 ORU Message Triage (Text)

### The Problem

Clinical staff receive **1000+ HL7 messages per day**. Critical values (K+ > 6.5, Troponin elevation) can be buried in routine results.

### The Solution: MedGemma Text Reasoning

MedGemma triages incoming lab results and clinical messages by urgency.

In [ ]:
from caremap.hl7_triage import load_sample_messages, triage_oru_message

messages = load_sample_messages()
stat_msg = [m for m in messages if m['expected_priority'] == 'STAT'][0]

print(f"Sample STAT Message: {stat_msg['message_id']}")
print(f"Patient: {stat_msg['patient']['age']}yo {stat_msg['patient']['gender']}")
print(f"Context: {stat_msg['clinical_context']}")
print(f"\nObservations:")
for obs in stat_msg['observations']:
    print(f"  - {obs['test_name']}: {obs['value']} {obs.get('units', '')} (ref: {obs['reference_range']})")

In [ ]:
import threading
import time

print("\nTriaging with MedGemma...")
print("(Single LLM call - typically takes 1-2 minutes on MPS)")

triage_result = {}
triage_error = [None]

def run_triage():
    try:
        triage_result.update(triage_oru_message(client, stat_msg))
    except Exception as e:
        triage_error[0] = e

thread = threading.Thread(target=run_triage)
thread.start()

with tqdm(desc="HL7 Triage (MedGemma inference)", unit="s", bar_format="{desc}: {elapsed} elapsed |{bar}|") as pbar:
    while thread.is_alive():
        time.sleep(1)
        pbar.update(1)

thread.join()
if triage_error[0]:
    raise triage_error[0]

result = triage_result
priority = result.get('priority', 'STAT')
print(f"\n{priority_emoji.get(priority, '')} Triage Result: {priority}")
print(f"Reason: {result.get('priority_reason', 'N/A')}")
print(f"Recommended Action: {result.get('recommended_action', 'N/A')}")

---

## Impact Summary

### One Model, Maximum Impact

| Module | Beneficiary | Impact |
|--------|-------------|--------|
| **Fridge Sheets** | Caregivers & Ayahs | Reduced confusion, fewer medication errors |
| **Radiology Triage** | Radiologists | Critical findings prioritized, faster diagnosis |
| **HL7 Triage** | Lab/Clinical Staff | Life-threatening values surfaced immediately |

### Why This Matters for India

- **Caregivers**: Rotating Ayahs can follow clear instructions without verbal handoffs
- **Radiologists**: Scarce specialists can focus on urgent cases first
- **Patients**: Better outcomes through timely intervention

### The Poster Effect: From Recall to Reference

> "The ayah changes every week. **How do you hand off care?**" — Dr. Gaurav Mishra

Printed fridge sheets fundamentally change how care is communicated:

| Without CareMap | With CareMap |
|-----------------|--------------|
| Outgoing ayah tries to remember all instructions | Outgoing ayah points to the poster on the fridge |
| New ayah relies on verbal handoff (incomplete, error-prone) | New ayah reads the poster and asks clarifying questions |
| Family must re-explain everything each week | Family walks new ayah through the printed pages once |
| Critical warnings get forgotten over time | Warnings stay visible: "Do NOT take with ibuprofen" |

**The goal is not perfect recall — it's reliable reference.** When the poster is on the fridge, the next Ayah doesn't need to memorize anything. They just need to read, follow, and ask when something doesn't make sense.

---

## Model Attribution

```bibtex
@article{sellergren2025medgemma,
  title={MedGemma Technical Report},
  author={Sellergren, Andrew and others},
  journal={arXiv preprint arXiv:2507.05201},
  year={2025}
}
```

---

**Interactive Demo:** [CareMap on HuggingFace Spaces](https://huggingface.co/spaces/codekunoichi/caremap-medgemma)

*CareMap - Making healthcare information accessible and actionable.*